In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.getcwd()
os.chdir("/content/drive/MyDrive/ch1")

Mounted at /content/drive


In [14]:
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Import tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

# Import other libraries
import cv2
from tensorflow.keras.applications.mobilenet import preprocess_input
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#for gan...
#!pip install visualkeras
#import visualkeras

#import imageio
#from PIL import Image
#from IPython.display import display
#import seaborn as sns
#import warnings
#import logging

#from tqdm import tqdm

2.14.0


In [3]:
# Load images from the 'items/' folder
temp = np.load('/content/drive/MyDrive/ch1/public_data.npz', allow_pickle = True)
img = temp["data"]
label = temp["labels"]
  # Normalize image pixel values to a float range [0, 1]
img = (img / 255).astype(np.float32)

# set 0,1 label
for i in range(len(label)):
    if(label[i] == 'healthy'):
        label[i] = 0
    else:
        label[i] = 1

# cleaning images from trol and shrek
ref_img = img[58]
ref_img2 = img[2150]
c = 0
c2 = 0
rm_indexes = []
rm2_indexes = []
for i in range(0, len(img)):
    deviation = np.mean(np.abs(ref_img - img[i]))
    deviation2 = np.mean(np.abs(ref_img2 - img[i]))
    if(deviation == 0.0):
        #print(i)
        c += 1
        rm_indexes.append(i)
    elif(deviation2 == 0.0):
        c2 += 1
        rm2_indexes.append(i)

clean_img = np.delete(img, rm_indexes + rm2_indexes, axis=0)
clean_label = np.delete(label, rm_indexes + rm2_indexes, axis=0)
print(clean_img.shape)



(5004, 96, 96, 3)


In [4]:
BATCH_SIZE = 32
#directory for saved models
saved_models_dir = "/content/drive/MyDrive/ch1"
#directory for checkpoints
Checkpoints_dir  = "/content/drive/MyDrive/ch1"

Train - VAlidation split

In [5]:
# Concatenate 'animals' and 'items' arrays along axis 0
X = clean_img
# Create labels: 1 for 'animals', 0 for 'items'
y = clean_label

y = keras.utils.to_categorical(y, 2) #one hot encodi

# Split data into train_val and test sets
#X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=1000, stratify=np.argmax(y,axis=1))
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=seed, test_size=.15, stratify=np.argmax(y,axis=1))

# Further split train_val into train and validation sets
#X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=1000, stratify=np.argmax(y_train_val,axis=1))

# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

X_train shape: (4253, 96, 96, 3), y_train shape: (4253, 2)
X_val shape: (751, 96, 96, 3), y_val shape: (751, 2)


AUGMENTED TRAINING SET

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # train generator with augmentation
train_image_gen  = ImageDataGenerator(rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      zoom_range=[0.5,1.5],
                                      shear_range=0.2,
                                      vertical_flip=True,
                                      horizontal_flip=True,
                                      fill_mode='reflect',
                                      )

# validation generator without augmentation

train_dataset = train_image_gen.flow(x = X_train, y = y_train, seed = 42, batch_size = BATCH_SIZE)




VGG16

In [8]:

conv_base  = tf.keras.applications.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224,3)
)
conv_base.trainable = False

58889256/58889256 [==============================] - 1s 0us/step


In [9]:
IMAGE_SHAPE = [96,96]
INPUT_SHAPE = (*IMAGE_SHAPE,3)
BATCH_SIZE = 16
SEED = 42


BUILD MODEL

In [22]:
inputs = tf.keras.Input(shape=INPUT_SHAPE)

x = layers.Resizing(224, 224, interpolation='bicubic', name='resizing')(inputs)
x = tf.keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)

x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(384, kernel_initializer=keras.initializers.HeUniform(seed=SEED))(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(0.3)(x)

x = layers.Dense(256, kernel_initializer=keras.initializers.HeUniform(seed=SEED))(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(2, activation='softmax', kernel_initializer=keras.initializers.GlorotUniform(seed=SEED))(x)

model_vgg16 = tf.keras.Model(inputs, outputs, name='vgg16')

callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=100, restore_best_weights=True, mode='auto')]
model_vgg16.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [ ]:
history = model_vgg16.fit(train_dataset,epochs=5000,validation_data=(X_val, y_val),callbacks=callbacks)


Epoch 1/5000
266/266 [==============================] - 2668s 10s/step - loss: 0.7259 - accuracy: 0.5899 - val_loss: 0.8295 - val_accuracy: 0.3808
Epoch 2/5000
106/266 [==========>...................] - ETA: 22:02 - loss: 0.7002 - accuracy: 0.5961